# LangChain - OutputParser와 구조화된 응답 처리

***환경 설정***

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from pprint import pprint

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

## 1. OutputParser 클래스

- `OutputParser`는 LLM 응답을 원하는 데이터 형식으로 변환하는 데 사용하는 구성 요소임
- `str`, `dict`, JSON, XML 등 여러 구조화된 형태로 파싱할 수 있음
- 파싱된 출력은 다른 시스템이나 프로세스와 연동하는 데 유용함

### 1\) JSONOutputParser

- `JSONOutputParser`는 LLM의 응답을 엄격한 JSON 형식으로 파싱하는 데 사용됨
- 출력값의 데이터 유효성 검증 및 일관된 스키마 보장에 유리함
- 일반적으로 LLM이 JSON 형식으로 응답하도록 `PromptTemplate`에 명시한 뒤, 해당 파서를 통해 결과를 구조화함
- 출력값을 바로 딕셔너리 형태로 변환해 사용할 수 있어, API 응답이나 다른 시스템과 연동 시 유요함

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing import List

# 관광지 정보를 위한 Pydantic 모델 정의
class TouristSpot(BaseModel):
    name: str = Field(description="관광명소 이름")
    location: str = Field(description="위치 (구/동 정보)")
    category: str = Field(description="카테고리 (궁궐/박물관/쇼핑 등)")
    highlights: List[str] = Field(description="주요 관람 포인트")

# JsonOutputParser 파서 설정
parser = JsonOutputParser(pydantic_object=TouristSpot)

# 파서의 포맷 지시사항 출력
pprint(parser.get_format_instructions())

('The output should be formatted as a JSON instance that conforms to the JSON '
 'schema below.\n'
 '\n'
 'As an example, for the schema {"properties": {"foo": {"title": "Foo", '
 '"description": "a list of strings", "type": "array", "items": {"type": '
 '"string"}}}, "required": ["foo"]}\n'
 'the object {"foo": ["bar", "baz"]} is a well-formatted instance of the '
 'schema. The object {"properties": {"foo": ["bar", "baz"]}} is not '
 'well-formatted.\n'
 '\n'
 'Here is the output schema:\n'
 '```\n'
 '{"properties": {"name": {"description": "관광명소 이름", "title": "Name", "type": '
 '"string"}, "location": {"description": "위치 (구/동 정보)", "title": "Location", '
 '"type": "string"}, "category": {"description": "카테고리 (궁궐/박물관/쇼핑 등)", '
 '"title": "Category", "type": "string"}, "highlights": {"description": "주요 관람 '
 '포인트", "items": {"type": "string"}, "title": "Highlights", "type": "array"}}, '
 '"required": ["name", "location", "category", "highlights"]}\n'
 '```')


In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 정의
prompt = PromptTemplate(
    template="""서울의 다음 관광명소에 대한 상세 정보를 제공해주세요.
{format_instructions}

관광지: {spot_name}
""",
    input_variables=["spot_name"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 프롬프트 출력
pprint(prompt.format(spot_name="경복궁"))

('서울의 다음 관광명소에 대한 상세 정보를 제공해주세요.\n'
 'The output should be formatted as a JSON instance that conforms to the JSON '
 'schema below.\n'
 '\n'
 'As an example, for the schema {"properties": {"foo": {"title": "Foo", '
 '"description": "a list of strings", "type": "array", "items": {"type": '
 '"string"}}}, "required": ["foo"]}\n'
 'the object {"foo": ["bar", "baz"]} is a well-formatted instance of the '
 'schema. The object {"properties": {"foo": ["bar", "baz"]}} is not '
 'well-formatted.\n'
 '\n'
 'Here is the output schema:\n'
 '```\n'
 '{"properties": {"name": {"description": "관광명소 이름", "title": "Name", "type": '
 '"string"}, "location": {"description": "위치 (구/동 정보)", "title": "Location", '
 '"type": "string"}, "category": {"description": "카테고리 (궁궐/박물관/쇼핑 등)", '
 '"title": "Category", "type": "string"}, "highlights": {"description": "주요 관람 '
 '포인트", "items": {"type": "string"}, "title": "Highlights", "type": "array"}}, '
 '"required": ["name", "location", "category", "highlights"]}\n'

In [13]:
# 실행
chain = prompt | llm | parser
result = chain.invoke({"spot_name": "경복궁"})

# 결과 출력
pprint(result)

{'category': '궁궐',
 'highlights': ['조선 시대의 대표적인 궁궐로서 한국 전통 건축의 아름다움을 감상할 수 있음',
                '근정전, 경회루, 사정전 등 주요 건물 관람',
                '한복 체험과 수문장 교대식 관람 가능',
                '경복궁 내 국립고궁박물관과 국립민속박물관 방문',
                '사계절마다 다른 아름다운 경관'],
 'location': '종로구 세종로',
 'name': '경복궁'}


### 2\) XMLOutputParser

- `XMLOutputParser`는 LLM의 응답을 계층적 구조를 갖는 XML 형식으로 파싱함
- XML은 노드 간의 관계 표현이 가능하여, 복잡한 데이터 구조나 문서형 응답을 표현할 때 효과적임
- 일반적인 JSON 보다 문서 중심의 구조나 메타데이터가 많은 응답을 다룰 때 유리함
- 내부적으로 XML 파싱을 위해 `defusedxml` 패키지를 사용하므로, 사전 설치가 필요함
- JSON에 비해 사용 빈도는 낮지만, RDF, 문서 포맷, 일부 산업용 스키마와 연계 시 유용하게 사용됨

In [14]:
!uv pip install defusedxml

Resolved 1 package in 155ms
Prepared 1 package in 54ms
Installed 1 package in 26ms
 + defusedxml==0.7.1


In [5]:
from langchain_core.output_parsers import XMLOutputParser

# XML 파서 설정
parser = XMLOutputParser(
    tags=["tourist_spot", "name", "location", "category", "highlights", "point"]
)

# 파서의 포맷 지시사항 출력
pprint(parser.get_format_instructions())

('The output should be formatted as a XML file.\n'
 '1. Output should conform to the tags below.\n'
 '2. If tags are not given, make them on your own.\n'
 '3. Remember to always open and close all the tags.\n'
 '\n'
 'As an example, for the tags ["foo", "bar", "baz"]:\n'
 '1. String "<foo>\n'
 '   <bar>\n'
 '      <baz></baz>\n'
 '   </bar>\n'
 '</foo>" is a well-formatted instance of the schema.\n'
 '2. String "<foo>\n'
 '   <bar>\n'
 '   </foo>" is a badly-formatted instance.\n'
 '3. String "<foo>\n'
 '   <tag>\n'
 '   </tag>\n'
 '</foo>" is a badly-formatted instance.\n'
 '\n'
 'Here are the output tags:\n'
 '```\n'
 "['tourist_spot', 'name', 'location', 'category', 'highlights', 'point']\n"
 '```')


In [7]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 정의
prompt = PromptTemplate(
    template="""서울의 다음 관광명소에 대한 상세 정보를 XML 형식으로 제공해주세요.
{format_instructions}

관광지: {spot_name}""",
    input_variables=["spot_name"],
    partial_variables={"format_instructions": parser.get_format_instructions()}  
)

# 프롬프트 출력
pprint(prompt.format(spot_name="경복궁"))

('서울의 다음 관광명소에 대한 상세 정보를 XML 형식으로 제공해주세요.\n'
 'The output should be formatted as a XML file.\n'
 '1. Output should conform to the tags below.\n'
 '2. If tags are not given, make them on your own.\n'
 '3. Remember to always open and close all the tags.\n'
 '\n'
 'As an example, for the tags ["foo", "bar", "baz"]:\n'
 '1. String "<foo>\n'
 '   <bar>\n'
 '      <baz></baz>\n'
 '   </bar>\n'
 '</foo>" is a well-formatted instance of the schema.\n'
 '2. String "<foo>\n'
 '   <bar>\n'
 '   </foo>" is a badly-formatted instance.\n'
 '3. String "<foo>\n'
 '   <tag>\n'
 '   </tag>\n'
 '</foo>" is a badly-formatted instance.\n'
 '\n'
 'Here are the output tags:\n'
 '```\n'
 "['tourist_spot', 'name', 'location', 'category', 'highlights', 'point']\n"
 '```\n'
 '\n'
 '관광지: 경복궁')


In [8]:
# 실행
chain = prompt | llm | parser
result = chain.invoke({"spot_name": "경복궁"})

# 결과 출력
pprint(result)

{'tourist_spot': [{'name': '경복궁'},
                  {'location': '서울특별시 종로구 사직로 161'},
                  {'category': '역사/문화유적'},
                  {'highlights': [{'point': '조선 왕조의 정궁으로서 한국 전통 건축의 대표적 예'},
                                  {'point': '근정전, 경회루 등 주요 건축물과 아름다운 정원'},
                                  {'point': '한복 체험 및 전통 문화 행사 참여 가능'},
                                  {'point': '광화문 광장과 인접해 있어 접근성 우수'}]}]}


## 2. 사용자 정의 Output Parser

- 기본 `OutputParser`로 처리하기 어려운 복잡한 출력 형식이나 도메인 특화된 요구사항에 대응하기 위해, 사용자 정의 파서를 직접 구현할 수 있음

In [12]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4.1-mini", model_provider="openai")

In [14]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿
step_prompt = PromptTemplate(
    template="""다음 텍스트에 대해서 작업을 순서대로 수행하세요:

    [텍스트]
    {text}

    [작업 순서]
    1. 텍스트를 1문장으로 요약
    2. 핵심 키워드 3개 추출
    3. 감정 분석 수행(긍정/부정/중립)

    [작업 결과]
    """,
    input_variables=["text"]
)

# 입력 텍스트
text = """
양자 컴퓨팅은 양자역학의 원리를 바탕으로 데이터를 처리하는 새로운 형태의 계산 방식이다.
기존의 고전적 컴퓨터는 0과 1로 이루어진 이진법(bit)을 사용하여 데이터를 처리하지만,
양자 컴퓨터는 양자 비트(큐비트, qubit)를 사용하여 훨씬 더 복잡하고 빠른 계산을 수행할 수 있다.

큐비트는 동시에 0과 1의 상태를 가질 수 있는 양자 중첩(superposition) 상태를 활용하며,
이를 통해 병렬 계산과 같은 고급 기능이 가능하다.
"""

### 1\) RunnableLambda 기반 방식

- LLM 응답에서 특정 키워드를 추출, 조건 분기 처리, 외부 함수 호출 등 고유한 후처리 로직을 삽입할 수 있음
- 고정된 데이터 구조 이외의 유연한 형식을 다루거나, 모델 출력의 후처리를 코드 기반으로 상세히 조절할 때 유용함

In [ ]:
from langchain_core.messages import AIMessage
from langchain_core.runnables import RunnableLambda
from typing import Dict

# 사용자 정의 파서
def custom_parser(ai_message: AIMessage) -> Dict:
    """모델 출력을 리스트 형태로 변환"""
    return ai_message.content.split('\n')

# 실행
chain = step_prompt | llm | RunnableLambda(custom_parser)
result = chain.invoke({"text": text})

# 결과 출력
pprint(result)

['1. 요약: 양자 컴퓨팅은 양자역학의 원리를 이용해 큐비트를 통한 중첩 상태로 기존 컴퓨터보다 훨씬 빠르고 복잡한 계산을 수행하는 새로운 '
 '계산 방식이다.  ',
 '2. 핵심 키워드: 양자 컴퓨팅, 큐비트, 중첩(superposition)  ',
 '3. 감정 분석: 중립']


### 2\) typing 기반 방식

- 출력 구조를 가볍게 명시하면서, 간단 JSON 응답을 기대할 때 사용함

In [ ]:
from typing import TypedDict, Annotated 

# 구조화된 출력 스키마
class AnalysisResult(TypedDict):
    """분석 결과 스키마"""
    summary: Annotated[str, ..., "핵심 요약"]   # ...은 필수 입력을 의미
    keywords: Annotated[list[str], ..., "주요 키워드"]
    sentiment: Annotated[str, ..., "긍정/부정/중립"]

structured_llm = llm.with_structured_output(AnalysisResult)

# 실행
chain = step_prompt | structured_llm
output = chain.invoke({"text": text})
pprint(output)

{'keywords': ['양자 컴퓨팅', '큐비트', '양자 중첩'],
 'sentiment': '중립',
 'summary': '양자 컴퓨팅은 양자역학 원리를 이용해 큐비트를 통해 고전 컴퓨터보다 훨씬 빠르고 복잡한 계산을 가능하게 하는 새로운 '
            '계산 방식이다.'}


### 3\) pydantic 기반 방식

- 데이터 타입 검증, 필수 항목 검사, 상세 오류 처리까지 가능한 견고한 방식임
- 구조가 명확한 응답을 기대할 수 있어, API 응답 처리, DB 저장, UI 렌더링 등과의 연동에 효과적임

In [16]:
from typing import List, Literal
from pydantic import BaseModel, Field

# 구조화된 출력 스키마
class AnalysisResult(BaseModel):
    """분석 결과 스키마"""
    summary: str = Field(...,  description="텍스트의 핵심 내용 요약")
    keywords: List[str] = Field(..., description="텍스트에서 추출한 주요 키워드")
    sentiment: Literal["긍정", "부정", "중립"] = Field(
        ..., 
        description="텍스트의 전반적인 감정 분석 결과"
    )

structured_llm = llm.with_structured_output(AnalysisResult)

# 실행
chain = step_prompt | structured_llm
output = chain.invoke({"text": text})
print(output.summary)
print(output.keywords)
print(output.sentiment)

양자 컴퓨팅은 양자 중첩 상태를 활용하는 큐비트를 통해 기존 고전 컴퓨터보다 더 복잡하고 빠른 계산을 가능하게 하는 새로운 계산 방식이다.
['양자 컴퓨팅', '큐비트', '양자 중첩']
긍정


## 3. [예제] 학습 도우미 만들기

***환경 설정***

In [ ]:
from dotenv import load_dotenv
load_dotenv()

### 1\) 퀴즈 생성 챗봇

In [ ]:
from typing import List
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 스키마 정의의
class QuizQuestion(BaseModel):
    """퀴즈 스키마"""
    question: str = Field(..., description="퀴즈 문제")
    options: List[str] = Field(..., description="보기 (4개)")
    correct_answer: int = Field(..., description="정답 번호 (1-4)")
    explanation: str = Field(..., description="정답 설명")


# 프롬프트 탬플릿
quiz_prompt = PromptTemplate(
    template="""다음 주제에 대한 퀴즈 문제를 만들어주세요:
    
주제: {topic}
난이도(상/중/하): {difficulty}

다음 조건을 만족하는 퀴즈를 생성해주세요:
1. 문제는 명확하고 이해하기 쉽게
2. 4개의 보기 제공
3. 정답과 오답은 비슷한 수준으로
4. 상세한 정답 설명 포함""",
    input_variables=["topic", "difficulty"]
)

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3)

# 구조화된 Outupt Parser 설정
structured_llm = llm.with_structured_output(QuizQuestion)

# 실행
chain = quiz_prompt | structured_llm
output = chain.invoke({"topic": "LangChain", "difficulty": "상"})

# 결과 출력
pprint(f"퀴즈 문제: {output.question}")
pprint(f"보기: {output.options}")
pprint(f"정답: {output.correct_answer}")
pprint(f"정답 설명: {output.explanation}")

"퀴즈 문제: LangChain에서 '체인(chain)'의 주요 역할은 무엇인가?"
("보기: ['여러 개의 LLM 호출을 순차적으로 연결하여 복잡한 작업을 수행한다.', '데이터베이스와의 연결을 관리한다.', '사용자 "
 "인터페이스를 구성하는 모듈이다.', '모델 학습을 위한 데이터 전처리를 담당한다.']")
'정답: 1'
("정답 설명: LangChain에서 '체인'은 여러 개의 언어 모델 호출을 순차적으로 연결하여 복잡한 작업을 수행하는 역할을 합니다. 이를 "
 '통해 단일 모델 호출로는 어려운 복합적인 작업을 단계별로 처리할 수 있습니다. 데이터베이스 연결이나 UI 구성, 데이터 전처리는 각각 '
 '다른 컴포넌트가 담당합니다.')


### 2\) 개념 설명 챗봇

In [26]:
from typing import List
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 스키마 정의의
class ConceptExplanation(BaseModel):
    """개념 설명 스키마"""
    topic: str = Field(..., description="주제")
    explanation: str = Field(..., description="개념 설명")
    examples: str = Field(..., description="사용 예시")
    related_concepts: List[str] = Field(..., description="관련된 개념 (4개)")

# 프롬프트 탬플릿
concept_prompt = PromptTemplate(
    template="""다음 주제에 대해 차근차근 설명해 주세요:
    
주제: {topic}
난이도(상/중/하): {difficulty}

다음을 차례대로 작성하세요:
1. 주제에 대한 개념 설명
2. 주제에 대한 사용 예시
3. 관련 개념 목록 (4개)
""",
    input_variables=["topic", "difficulty"]
)

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3)

# 구조화된 Outupt Parser 설정
structured_llm = llm.with_structured_output(ConceptExplanation)

# 실행
chain = quiz_prompt | structured_llm
output = chain.invoke({"topic": "LangChain", "difficulty": "하"})

# 결과 출력
pprint(f"주제: {output.topic}")
pprint(f"설명: {output.explanation}")
pprint(f"예시: {output.examples}")
pprint(f"관련 개념: {output.related_concepts}")

'주제: LangChain'
('설명: LangChain은 언어 모델을 활용하여 다양한 애플리케이션을 개발할 수 있도록 돕는 프레임워크입니다. 주로 자연어 처리 작업을 '
 '쉽게 연결하고 확장할 수 있게 설계되었습니다.')
'예시: 예를 들어, LangChain을 사용하면 텍스트 요약, 질문 응답, 대화형 에이전트 등을 쉽게 구현할 수 있습니다.'
"관련 개념: ['자연어 처리', '언어 모델', '프레임워크', 'AI 애플리케이션']"
